In [1]:
import pandas as pd
import findspark
findspark.init("/opt/spark")
from pyspark.sql.functions import explode, from_json, explode_outer
from pyspark.sql.functions import col, max, min
from pyspark.sql.types import *
from delta.tables import *

In [2]:
accessKeyId='dataops9'
secretAccessKey='Ankara06'

# create a SparkSession
spark = SparkSession.builder \
.appName("schemas") \
.master("local[2]") \
.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.0,io.delta:delta-core_2.12:2.4.0") \
.config("fs.s3a.access.key", accessKeyId) \
.config("fs.s3a.secret.key", secretAccessKey) \
.config("fs.s3a.path.style.access", True) \
.config("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
.config("fs.s3a.endpoint", "http://minio:9000") \
.config("spark.sql.debug.maxToStringFields", 1000) \
.getOrCreate()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b1799de2-368f-4659-b0dd-ce1ad367a4e1;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 409ms :: artifacts dl 10ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [default]
	------------------------------------

1.cast tablosu 

In [3]:
cast_schema = StructType([
    StructField("movie_id", StringType(), nullable=True),
    StructField("title", StringType(), nullable=True),
    StructField("cast_id", IntegerType(), nullable=True),
    StructField("character", StringType(), nullable=True),
    StructField("credit_id", StringType(), nullable=True),
    StructField("gender", IntegerType(), nullable=True),
    StructField("id", IntegerType(), nullable=True),
    StructField("name", StringType(), nullable=True)
])

cast_table = spark.createDataFrame([], cast_schema)
cast_table.write.format("delta").mode("overwrite").save('s3a://tmdb-silver/credits_cast')

24/03/23 04:47:36 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


2.crew tablosu

In [4]:
crew_schema = StructType([
    StructField("movie_id", StringType(), nullable=True),
    StructField("title", StringType(), nullable=True),
    StructField("credit_id", StringType(), nullable=True),
    StructField("department", StringType(), nullable=True),
    StructField("gender", IntegerType(), nullable=True),
    StructField("id", IntegerType(), nullable=True),
    StructField("job", StringType(), nullable=True),
    StructField("name", StringType(), nullable=True)
])

crew_table = spark.createDataFrame([], crew_schema)
crew_table.write.format("delta").mode("overwrite").save('s3a://tmdb-silver/credits_crew')

3.movies tablosu

In [5]:
movies_schema = StructType([
    StructField("movie_id", StringType(), nullable=True),
    StructField("title", StringType(), nullable=True),
    StructField("budget", DoubleType(), nullable=True),
    StructField("homepage", StringType(), nullable=True),
    StructField("original_language", StringType(), nullable=True),
    StructField("original_title", StringType(), nullable=True),
    StructField("overview", StringType(), nullable=True),
    StructField("popularity", FloatType(), nullable=True),
    StructField("release_date", DateType(), nullable=True),
    StructField("revenue", DoubleType(), nullable=True),
    StructField("runtime", IntegerType(), nullable=True),
    StructField("status", StringType(), nullable=True),
    StructField("tagline", StringType(), nullable=True),
    StructField("vote_average", FloatType(), nullable=True),
    StructField("vote_count", IntegerType(), nullable=True)
])

movies_table = spark.createDataFrame([], movies_schema)
movies_table.write.format("delta").mode("overwrite").save('s3a://tmdb-silver/movies')

4.genres tablosu

In [6]:
genres_schema = StructType([
    StructField("movie_id", StringType(), nullable=True),
    StructField("id", IntegerType(), nullable=True),
    StructField("name", StringType(), nullable=True)
])

genres_table = spark.createDataFrame([], genres_schema)
genres_table.write.format("delta").mode("overwrite").save('s3a://tmdb-silver/genres')

5.keywords tablosu

In [7]:
keywords_schema = StructType([
    StructField("movie_id", StringType(), nullable=True),
    StructField("id", IntegerType(), nullable=True),
    StructField("name", StringType(), nullable=True)
])
keywords_table = spark.createDataFrame([], keywords_schema)
keywords_table.write.format("delta").mode("overwrite").save('s3a://tmdb-silver/keywords')

6.production_companies tablosu

In [8]:
production_companies_schema = StructType([
    StructField("movie_id", StringType(), nullable=True),
    StructField("id", IntegerType(), nullable=True),
    StructField("name", StringType(), nullable=True)
])
production_companies_table = spark.createDataFrame([], production_companies_schema)
production_companies_table.write.format("delta").mode("overwrite").save('s3a://tmdb-silver/production_companies')

7.production_countries tablosu

In [9]:
production_countries_schema = StructType([
    StructField("movie_id", StringType(), nullable=True),
    StructField("iso_3166_1", StringType(), nullable=True),
    StructField("name", StringType(), nullable=True)
])
production_countries_table = spark.createDataFrame([], production_countries_schema)
production_countries_table.write.format("delta").mode("overwrite").save('s3a://tmdb-silver/production_countries')

8.spoken_languages tablosu

In [10]:
spoken_languages_schema = StructType([
    StructField("movie_id", StringType(), nullable=True),
    StructField("iso_639_1", StringType(), nullable=True),
    StructField("name", StringType(), nullable=True)
])
spoken_languages_table = spark.createDataFrame([], spoken_languages_schema)
spoken_languages_table.write.format("delta").mode("overwrite").save('s3a://tmdb-silver/spoken_languages')

24/03/23 08:45:58 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from cc0b8b1f1e73:35534 in 10000 milliseconds
